*Note that this workflow task should be executed against a Serverless SQL Warehouse*

# FHIR Streaming Bronze Ingestion 
***

In [0]:
DECLARE OR REPLACE VARIABLE catalog_use STRING DEFAULT 'fhir_workshop';
DECLARE OR REPLACE VARIABLE schema_use STRING DEFAULT REPLACE(SPLIT(current_user(), '@')[0], '.', '_');

In [0]:
SET VARIABLE catalog_use = :catalog_use; 
SET VARIABLE schema_use = :schema_use; 

In [0]:
USE IDENTIFIER(catalog_use || "." || schema_use);

In [0]:
select current_catalog(), current_schema();

In [0]:
DECLARE OR REPLACE VARIABLE volume_path STRING DEFAULT "/Volumes/" || catalog_use || "/" || schema_use || "/landing/";

SET VARIABLE volume_path = (
  CASE when catalog_use = 'fhir_workshop' and schema_use LIKE "synthea%" THEN "/Volumes/" || catalog_use || "/" || schema_use || "/synthetic_files_raw/output/fhir/"
  ELSE "/Volumes/" || catalog_use || "/" || schema_use || "/landing/" END
);

SELECT volume_path;

In [0]:
DECLARE OR REPLACE VARIABLE create_streaming_bronze_stmnt STRING;

SET VARIABLE create_streaming_bronze_stmnt = "
CREATE OR REFRESH STREAMING TABLE fhir_bronze (
  file_metadata STRUCT<
    file_path: STRING,
    file_name: STRING,
    file_size: BIGINT,
    file_block_start: BIGINT,
    file_block_length: BIGINT,
    file_modification_time: TIMESTAMP
  > NOT NULL COMMENT 'Original meta date of the file ingested from the volume.'
  ,ingest_time TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP() COMMENT 'The date timestamp the file was ingested.'
  ,bundle_uuid STRING NOT NULL COMMENT 'Unique identifier for the FHIR bundle.'
  ,value STRING COMMENT 'Original JSON record ingested from the volume as a full text string value.'
)
COMMENT 'Ingest FHIR JSON records as Full Text STRING'
TBLPROPERTIES (
  'delta.enableChangeDataFeed' = 'true',
  'delta.enableDeletionVectors' = 'true',
  'delta.enableRowTracking' = 'true',
  'quality' = 'bronze'
)
AS SELECT
  _metadata as file_metadata
  ,uuid() as bundle_uuid
  ,* 
FROM STREAM read_files(
  '" || volume_path || "'
  ,format => 'text'
  ,wholeText => true
)
";

SELECT create_streaming_bronze_stmnt;

In [0]:
EXECUTE IMMEDIATE create_streaming_bronze_stmnt;

In [0]:
CREATE OR REFRESH STREAMING TABLE fhir_bronze_variant (
  bundle_uuid STRING NOT NULL COMMENT 'Unique identifier for the FHIR bundle.'
  ,ingest_time TIMESTAMP NOT NULL COMMENT 'The date timestamp the file was ingested.'
  ,file_metadata STRUCT<
    file_path: STRING,
    file_name: STRING,
    file_size: BIGINT,
    file_block_start: BIGINT,
    file_block_length: BIGINT,
    file_modification_time: TIMESTAMP
  > NOT NULL COMMENT 'Original meta date of the file ingested from the volume.'
  ,fhir VARIANT COMMENT 'Original JSON record fully parsed as a variant data type.'
)
COMMENT 'Evaluate FHIR JSON records as VARIANT'
TBLPROPERTIES (
  'delta.enableChangeDataFeed' = 'true'
  ,'delta.enableDeletionVectors' = 'true' 
  ,'delta.enableRowTracking' = 'true'
  ,'quality' = 'bronze'
  ,'pipelines.channel' = 'PREVIEW'
  ,'delta.feature.variantType-preview' = 'supported'
)
AS SELECT
  bundle_uuid
  ,ingest_time
  ,file_metadata
  ,try_parse_json(value) as fhir 
FROM STREAM fhir_bronze;